<div align="center">
    <H1>Aprendizado de Máquina - Trabalho Final</H1>
    <H3>Prof.º Daniel Roberto Cassar</H3> 
</div>

<br>

<div align="right">
    <H3>Guilda: Carcajus</H3>
    <H4>Aniel Souza Ribeiro Neto</H4>
    <H4>Caio ... </H4>
    <H4>Joaquim ...</H4>
</div>

<br> 

<div align="center">
    <img src="Imagens/carcajus_bg.png" width="500"/>
</div>